In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [2]:
epsilons = [0, .05, .1, .15, .2, .25, .3]
pretrainedModel = '.datasets/lenet_mnist_model.pth'
useCuda = False

In [3]:
# LeNet Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
testLoader = torch.utils.data.DataLoader(
    datasets.MNIST('./datasets', train=False, download=True, transform = transforms.Compose([
        transforms.ToTensor()
    ])), batch_size=1, shuffle=True
)

0it [00:00, ?it/s]

100%|█████████▉| 9879552/9912422 [00:20<00:00, 427430.06it/s]

Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz



0it [00:00, ?it/s]


 57%|█████▋    | 16384/28881 [00:00<00:00, 144767.37it/s]
32768it [00:00, 95728.31it/s]                            
0it [00:00, ?it/s]

Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz



 95%|█████████▍| 1564672/1648877 [00:03<00:00, 784344.93it/s]
1654784it [00:04, 412604.34it/s]                             
0it [00:00, ?it/s]

Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz



  0%|          | 0/4542 [00:00<?, ?it/s]
8192it [00:00, 31253.20it/s]            

Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [8]:
device = torch.device("cuda" if (useCuda and torch.cuda.is_available()) else "cpu")

In [9]:
# Initialize the network
model = Net().to(device)

In [10]:
# Load the pretrained model
model.load_state_dict(torch.load(pretrainedModel, map_location='cpu'))

In [11]:
# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

## FGSM Attack